In [1]:
# Importing the libraries
import pandas as pd
import numpy as np
import os
# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
cwd = os.getcwd() 
folder_path = os.path.join(cwd, 'RawData')

In [5]:
# reading csv files and storing as a dataframe
df_abgevents = pd.read_csv(os.path.join(folder_path,"ABGevents.csv"))
df_calc_death = pd.read_csv(os.path.join(folder_path,"calc_deaths.csv"))
df_referral = pd.read_csv(os.path.join(folder_path,"referrals.csv"))
df_culture_events = pd.read_csv(os.path.join(folder_path,"CultureEvents.csv"))
df_fluid_balance_events = pd.read_csv(os.path.join(folder_path,"FluidBalanceEvents.csv"))
df_hemo=pd.read_csv(os.path.join(folder_path,"HemoEvents.csv"))
df_serology=pd.read_csv(os.path.join(folder_path,"SerologyEvents.csv"))
df_chem=pd.read_csv(os.path.join(folder_path,"ChemistryEvents.csv"))
df_cbc=pd.read_csv(os.path.join(folder_path,"CBCEvents.csv"))

### Referral DataFrame Cleaning:

In [6]:
# check number of rows of referral dataframe
df_abgevents.shape[0]

2288828

In [7]:
# looking at the data
df_referral.head(5)

,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
0,OPO1,OPO1_P320866,62.0,M,White / Caucasian,OPO1_H23456,False,NaN,Head Trauma,NaN,...,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OPO1,OPO1_P549364,14.0,F,White / Caucasian,OPO1_H11908,False,NaN,NaN,NaN,...,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OPO1,OPO1_P536997,55.0,M,White / Caucasian,OPO1_H23111,False,NaN,CVA/Stroke,ICH/Stroke,...,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OPO1,OPO1_P463285,48.0,F,Black / African American,OPO1_H26589,False,NaN,Anoxia,Cardiovascular,...,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OPO1,OPO1_P284978,80.0,F,White / Caucasian,OPO1_H5832,False,NaN,NaN,NaN,...,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# checking the information of the dataframe
df_referral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133101 entries, 0 to 133100
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   OPO                     133101 non-null  object 
 1   PatientID               133101 non-null  object 
 2   Age                     133017 non-null  float64
 3   Gender                  133040 non-null  object 
 4   Race                    133101 non-null  object 
 5   HospitalID              133101 non-null  object 
 6   brain_death             133101 non-null  bool   
 7   Cause_of_Death_OPO      32396 non-null   object 
 8   Cause_of_Death_UNOS     103283 non-null  object 
 9   Mechanism_of_Death      98533 non-null   object 
 10  Circumstances_of_Death  98588 non-null   object 
 11  ABO_BloodType           18201 non-null   object 
 12  ABO_Rh                  8743 non-null    object 
 13  HeightIn                98500 non-null   float64
 14  WeightKg            

In [10]:
# List of columns to convert data type from object to timestamp
columns_to_convert = ['time_asystole', 'time_brain_death', 'time_referred', 'time_approached','time_authorized','time_procured']

# Apply pd.to_datetime to each column
for col in columns_to_convert:
    df_referral[col] = pd.to_datetime(df_referral[col], errors='coerce')

#List of columns to convert data type from float to int
col_toConvert_int = ['Procured_Year','Age']
# Apply astype() to convert
for colint in col_toConvert_int:
    df_referral[colint] = df_referral[colint].astype('Int64')

In [11]:
# checking for duplicate records 
print("Duplicate Rows:\n", df_referral.duplicated().sum())

Duplicate Rows:
 0


In [13]:
# Checking for the value counts of 'mechanism_of_death' column for cleanup
df_referral['Mechanism_of_Death'].value_counts()

Mechanism_of_Death
Cardiovascular                       27267
Natural Causes                       25882
ICH/Stroke                           16476
None of the Above                     8757
Blunt Injury                          8313
Drug Intoxication                     3285
Gun Shot Wound                        1992
Asphyxiation                          1939
Intracranial Hemmorrhage / Stroke     1311
Death from Natural Causes              890
Seizure                                775
Drug / Intoxication                    546
Drowning                               507
Gunshot Wound                          297
Other                                  112
Stab                                    76
Electrical                              51
Sudden Infant Death                     41
None of the above                       16
Name: count, dtype: int64

In [14]:
# Created dictionary mapping to replace similar values
replace_dict = {
    'Gun Shot Wound': 'Gunshot Wound',
    'Drug / Intoxication': 'Drug Intoxication',
    'None of the Above': 'None of the above',
    'ICH/Stroke':'Intracranial Hemmorrhage / Stroke',
    'Death from Natural Causes':'Natural Causes'
}

df_referral['Mechanism_of_Death'].replace(replace_dict, inplace=True)

In [15]:
# This step shows the updated value counts for 'mechanism_of_death' column
df_referral['Mechanism_of_Death'].value_counts()

Mechanism_of_Death
Cardiovascular                       27267
Natural Causes                       26772
Intracranial Hemmorrhage / Stroke    17787
None of the above                     8773
Blunt Injury                          8313
Drug Intoxication                     3831
Gunshot Wound                         2289
Asphyxiation                          1939
Seizure                                775
Drowning                               507
Other                                  112
Stab                                    76
Electrical                              51
Sudden Infant Death                     41
Name: count, dtype: int64

In [16]:
# Checking for the value counts of 'circumstances_of_death' column for cleanup
df_referral['Circumstances_of_Death'].value_counts() 

Circumstances_of_Death
Natural Causes                60700
None of the Above             17859
Accident, Non-MVA              6446
MVA                            5045
Death from Natural Causes      3470
Suicide                        2282
Homicide                       1293
Motor Vehicle Accident          481
Non-Motor Vehicle Accident      336
Alleged Suicide                 293
Alleged Homicide                141
Child Abuse                     127
Other                            95
Alleged Child Abuse              20
Name: count, dtype: int64

In [18]:
# Created dictionary mapping to replace similar values
replace_dict_cod = {
    'Natural Causes': 'Death from Natural Causes',
    'Accident, Non-MVA': 'Non-Motor Vehicle Accident',
    'MVA': 'Motor Vehicle Accident'   
}

df_referral['Circumstances_of_Death'].replace(replace_dict_cod, inplace=True)

In [19]:
# This step shows the updated value counts for 'circumstances_of_death' column
df_referral['Circumstances_of_Death'].value_counts()

Circumstances_of_Death
Death from Natural Causes     64170
None of the Above             17859
Non-Motor Vehicle Accident     6782
Motor Vehicle Accident         5526
Suicide                        2282
Homicide                       1293
Alleged Suicide                 293
Alleged Homicide                141
Child Abuse                     127
Other                            95
Alleged Child Abuse              20
Name: count, dtype: int64

In [20]:
df_referral['time_procured'].value_counts()

time_procured
2035-04-28 16:52:00    2
2032-11-07 21:59:00    2
2036-03-27 17:52:00    2
2031-04-19 19:54:00    2
2035-03-14 07:18:00    1
                      ..
2031-01-02 21:50:00    1
2030-05-15 18:04:00    1
2032-07-09 03:39:00    1
2032-10-05 01:03:00    1
2036-10-30 17:23:00    1
Name: count, Length: 7901, dtype: int64

In [23]:
# Checking bloodtype and rh column to see the count of not null values before merging both the columns
df_referral[(df_referral['ABO_BloodType'].notna() & df_referral['ABO_Rh'].notna())].shape

(8739, 38)

In [24]:
# Code to merge both the columns when the values are not null, This step ignores the cases where bloodtype is null and rh is not null
df_referral.loc[df_referral['ABO_BloodType'].notna() & df_referral['ABO_Rh'].notna(), 'ABO_BloodType'] = (
    df_referral['ABO_BloodType'] + '-' + df_referral['ABO_Rh']
)

In [26]:
# Checking to verify the unique values after merging the columns
df_referral['ABO_BloodType'].unique()

array([nan, 'O-Positive', 'A1-Positive', 'A-Positive', 'A2-Positive',
       'A1-Negative', 'A2B-Positive', 'B-Positive', 'O-Negative',
       'A1B-Positive', 'AB-Positive', 'B-Negative', 'A1B-Negative',
       'A-Negative', 'AB-Negative', 'A2-Negative', 'A', 'O', 'B', 'A1',
       'AB', 'A2B-Negative', 'A2B', 'A2', 'A1B', 'A1-Positive ',
       'A-Positive ', 'B-Positive ', 'A-Negative ', 'B-Negative ',
       'AB-Positive ', 'O-Positive ', 'A1-Negative ', 'O-Negative ',
       'AB-Negative ', 'A1B-Positive ', 'A2-Positive ', 'A2-Negative ',
       'A2B-Positive ', 'A1B-Negative ', 'A2B-Negative '], dtype=object)

In [27]:
# Deleting the columns 'ABO_Rh' and 'Referral_DayofWeek'. 'abo_rh' values are merged so its duplicate. 
#Dropped 'Referral_DayofWeek' column as we did not find any significance towards our objective.

df_referral.drop(columns=['ABO_Rh','Referral_DayofWeek'], inplace= True)

In [30]:
df_referral['WeightKg'].max()

9101.0

In [35]:
df_referral.query('WeightKg > 650')

,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
9882,OPO1,OPO1_P26310,55,F,Hispanic,OPO1_H4303,False,NaN,NaN,NaN,...,2020,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28246,OPO1,OPO1_P32277,46,F,White / Caucasian,OPO1_H20923,False,NaN,Other,NaN,...,2021,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31121,OPO1,OPO1_P836782,40,M,White / Caucasian,OPO1_H8351,False,NaN,Head Trauma,NaN,...,2020,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54984,OPO3,OPO3_P499799,47,F,White / Caucasian,OPO3_H6081,False,"Respiratory - Other, specify",Other,None of the above,...,2021,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85221,OPO4,OPO4_P752787,73,M,White / Caucasian,OPO4_H33509,False,NaN,ICB / ICH,Intracranial Hemmorrhage / Stroke,...,2020,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106820,OPO5,OPO5_P249805,15,M,White / Caucasian,OPO5_H10831,False,NaN,Head Trauma,Blunt Injury,...,2020,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111858,OPO6,OPO6_P327644,80,M,White / Caucasian,OPO6_H2155,False,Sepsis,Other,Natural Causes,...,2017,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119824,OPO6,OPO6_P371686,73,F,White / Caucasian,OPO6_H8991,False,"Other, specify",Anoxia,Natural Causes,...,2015,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120226,OPO6,OPO6_P578203,27,M,Black / African American,OPO6_H11154,True,NaN,Anoxia,Cardiovascular,...,2015,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123267,OPO6,OPO6_P721934,38,M,White / Caucasian,OPO6_H22175,False,NaN,Anoxia,Drug Intoxication,...,2016,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Converting all weight values above 650kg to null. Since normal weight of person could not exceed more than that as per our search.
df_referral['WeightKg'] = df_referral['WeightKg'].apply(lambda x: np.nan if x > 650 else x)

In [39]:
# rounding the weight column values to 2 decimal places
df_referral['WeightKg'] = df_referral['WeightKg'].round(2)

In [40]:
df_referral['HeightIn'].max()

1752.0

In [41]:
df_referral.query('HeightIn > 108')

,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
67,OPO1,OPO1_P189494,73,M,White / Caucasian,OPO1_H28513,False,NaN,Anoxia,Cardiovascular,...,2015,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,OPO1,OPO1_P266640,69,F,White / Caucasian,OPO1_H1793,False,NaN,Anoxia,None of the above,...,2016,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,OPO1,OPO1_P587625,65,F,Black / African American,OPO1_H28513,False,NaN,Head Trauma,Intracranial Hemmorrhage / Stroke,...,2015,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707,OPO1,OPO1_P111783,44,M,White / Caucasian,OPO1_H30962,False,NaN,NaN,NaN,...,2021,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
810,OPO1,OPO1_P527371,77,F,Hispanic,OPO1_H28513,False,NaN,NaN,NaN,...,2021,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110053,OPO5,OPO5_P122278,54,F,Black / African American,OPO5_H2402,False,NaN,NaN,NaN,...,2021,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110170,OPO5,OPO5_P934760,65,M,White / Caucasian,OPO5_H12973,False,RES - other,NaN,NaN,...,2021,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111295,OPO6,OPO6_P195992,68,M,White / Caucasian,OPO6_H14168,False,NaN,CVA/Stroke,Intracranial Hemmorrhage / Stroke,...,2015,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114281,OPO6,OPO6_P84410,42,M,White / Caucasian,OPO6_H8991,False,"Cardiac - Other, specify",Anoxia,Natural Causes,...,2015,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Converting all height values above 108inch to null. Since height of person can not exceed more than that as per our search.
df_referral['HeightIn'] = df_referral['HeightIn'].apply(lambda x: np.nan if x > 108 else x)

In [43]:
# rounding the height column values to 2 decimal places
df_referral['HeightIn'] = df_referral['HeightIn'].round(2)

In [44]:
# The process of procurement between January 1, 2015 and December 31, 2021. We noticed outliers before and after the mentioned date 
# in 'procured_year' column.- 10 rows
df_referral.loc[(df_referral['Procured_Year'] < 2015) | (df_referral['Procured_Year'] > 2021)]

,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
33285,OPO2,OPO2_P754522,45,M,Black / African American,OPO2_H5806,True,NaN,Anoxia,Drug Intoxication,...,2021,2022,NaN,Transplanted,NaN,NaN,NaN,NaN,NaN,NaN
35187,OPO2,OPO2_P105623,66,M,White / Caucasian,OPO2_H11956,True,NaN,Head Trauma,Blunt Injury,...,2021,2022,Recovered for Research,Transplanted,Transplanted,Transplanted,NaN,NaN,NaN,NaN
35334,OPO2,OPO2_P888977,62,F,White / Caucasian,OPO2_H11956,True,NaN,Anoxia,Drug Intoxication,...,2021,2022,NaN,NaN,Recovered for Transplant but not Transplanted,Recovered for Transplant but not Transplanted,NaN,NaN,NaN,NaN
45356,OPO2,OPO2_P691827,36,M,White / Caucasian,OPO2_H15401,True,NaN,Anoxia,Asphyxiation,...,2021,2022,Recovered for Research,Transplanted,Transplanted,Transplanted,NaN,NaN,Recovered for Research,Recovered for Research
49512,OPO3,OPO3_P893473,21,M,White / Caucasian,OPO3_H4291,False,NaN,NaN,NaN,...,2015,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50300,OPO3,OPO3_P87321,31,F,White / Caucasian,OPO3_H4291,False,NaN,NaN,NaN,...,2015,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108156,OPO5,OPO5_P437885,49,M,White / Caucasian,OPO5_H6087,False,An - other,Anoxia,Cardiovascular,...,2021,2022,NaN,NaN,Transplanted,Transplanted,NaN,NaN,NaN,NaN
109893,OPO5,OPO5_P544793,71,F,White / Caucasian,OPO5_H6087,True,NaN,Cerebrovascular / Stroke,Intracranial Hemmorrhage / Stroke,...,2021,2022,NaN,Transplanted,Recovered for Research,Recovered for Research,NaN,NaN,NaN,Recovered for Research
113737,OPO6,OPO6_P512017,1,F,White / Caucasian,OPO6_H6602,False,NaN,Anoxia,Drowning,...,2021,2022,NaN,NaN,Transplanted,Transplanted,NaN,NaN,NaN,NaN
117329,OPO6,OPO6_P319706,46,M,White / Caucasian,OPO6_H16901,True,NaN,Head Trauma,Blunt Injury,...,2021,2022,NaN,Transplanted,Transplanted,Transplanted,NaN,NaN,NaN,NaN


In [45]:
# Changing the outliers to null in 'procured_year' column
df_referral.loc[(df_referral['Procured_Year'] < 2015) | (df_referral['Procured_Year'] > 2021), 
       'Procured_Year'] = np.nan

In [46]:
# Procured step is before transplant so when transplant column has value True, procured has to be True. There is one patient who
# does not satify the condition

df_referral.loc[(df_referral['transplanted'] == True) & (df_referral['procured'] == False)]

,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
20119,OPO1,OPO1_P188776,60,M,Hispanic,OPO1_H26589,False,ICB / ICH,Other,Intracranial Hemmorrhage / Stroke,...,2015,<NA>,NaN,NaN,NaN,Transplanted,NaN,NaN,NaN,NaN


In [47]:
# Updating procured to True based on the condition for the above patient
df_referral.loc[(df_referral['transplanted'] == True) & (df_referral['procured'] == False), 
       'procured'] = True

In [48]:
# When procured_year column has year, we assume that the procurement happened so the procured column has to be True. 
# There are 39 patients who does not satisfy that condition.
df_referral.loc[(df_referral['Procured_Year'].notna()) & (df_referral['procured'] == False)]

,OPO,PatientID,Age,Gender,Race,HospitalID,brain_death,Cause_of_Death_OPO,Cause_of_Death_UNOS,Mechanism_of_Death,...,Referral_Year,Procured_Year,outcome_heart,outcome_liver,outcome_kidney_left,outcome_kidney_right,outcome_lung_left,outcome_lung_right,outcome_intestine,outcome_pancreas
385,OPO1,OPO1_P746312,26,M,Black / African American,OPO1_H27265,True,NaN,Anoxia,Cardiovascular,...,2017,2017,NaN,NaN,NaN,Recovered for Research,NaN,NaN,NaN,NaN
1181,OPO1,OPO1_P194150,0,M,Hispanic,OPO1_H15318,False,NaN,NaN,NaN,...,2017,2017,Recovered for Research,Recovered for Research,NaN,NaN,Recovered for Research,Recovered for Research,NaN,Recovered for Research
2139,OPO1,OPO1_P413576,79,F,White / Caucasian,OPO1_H21231,True,"Cardiac - Other, specify",Other,Intracranial Hemmorrhage / Stroke,...,2016,2016,NaN,Recovered for Transplant but not Transplanted,NaN,NaN,NaN,Recovered for Research,NaN,NaN
2829,OPO1,OPO1_P716240,67,F,Black / African American,OPO1_H22406,True,NaN,CVA/Stroke,Intracranial Hemmorrhage / Stroke,...,2018,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4285,OPO1,OPO1_P602137,64,M,White / Caucasian,OPO1_H30669,True,NaN,CVA/Stroke,Intracranial Hemmorrhage / Stroke,...,2016,2016,NaN,NaN,Recovered for Research,Recovered for Research,NaN,NaN,NaN,NaN
8350,OPO1,OPO1_P315126,67,M,White / Caucasian,OPO1_H475,True,NaN,Anoxia,Cardiovascular,...,2018,2018,Recovered for Research,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9470,OPO1,OPO1_P800088,39,M,White / Caucasian,OPO1_H25074,True,NaN,Anoxia,Cardiovascular,...,2019,2019,Recovered for Research,Recovered for Transplant but not Transplanted,NaN,NaN,NaN,NaN,NaN,NaN
11065,OPO1,OPO1_P307722,58,F,White / Caucasian,OPO1_H4959,True,NaN,Anoxia,Cardiovascular,...,2019,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11755,OPO1,OPO1_P159786,47,M,White / Caucasian,OPO1_H23111,True,NaN,CVA/Stroke,Intracranial Hemmorrhage / Stroke,...,2015,2015,NaN,NaN,NaN,NaN,Recovered for Research,Recovered for Research,NaN,Recovered for Research
12844,OPO1,OPO1_P311956,0,F,White / Caucasian,OPO1_H26720,False,Fetal Demise,Other,Natural Causes,...,2016,2016,NaN,NaN,NaN,NaN,Recovered for Research,Recovered for Research,NaN,Recovered for Research


In [49]:
# Updating procured to True when procured_year column has year value.
df_referral.loc[(df_referral['Procured_Year'].notna()) & (df_referral['procured'] == False), 
       'procured'] = True